# Sentiment Analysis of Amazone Baby Products

This project analyzes and develop a model to analyze and predict the sentiment given the costomer review for an amazon baby product. Turicreate is used for this work. The data set is highly biased towards the positive sentiment, so model is improved, by using only selected features for training the model.

## Exploring the Data

In [1]:
import turicreate as tc
import pandas as pd

In [2]:
products = tc.SFrame.read_csv('amazon_baby.csv')
products.head(5)

Finished parsing file /home/jovyan/work/Week 3/amazon_baby.csv

Parsing completed. Parsed 100 lines in 0.964432 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105270 lines. Lines per second: 83847.9

Finished parsing file /home/jovyan/work/Week 3/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 1.44833 secs.

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5


### Examining the most reviewed product

In [3]:
products.groupby('name',operations={'count':tc.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


In [4]:
giraffe = products[products['name'] == 'Vulli Sophie the Giraffe Teether']
giraffe.head(3)

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1


In [5]:
giraffe['rating'].show()

Materializing SArray

In [6]:
products['rating'].show()

Materializing SArray


## Modeling



In [7]:
products = products[products['rating']!= 3] #igonore 3 star ratings
products['sentiment'] = products['rating'] >= 4 # if 4 or 5 : positive, otherwise negative
products.head(3)

name,review,rating,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,1


In [8]:
products['sentiment'].show()

Materializing SArray

In [9]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
products['word_count'] = tc.text_analytics.count_words(products['review'])

In [10]:
def count_words(wc_dict,word):
    count = 0
    if word in wc_dict:
            count = wc_dict[word]
    return count

In [11]:
for word in selected_words:
    products[word]= products['word_count'].apply(lambda x: count_words(x,word))

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [12]:
products.head(3)

Columns:
	name	str
	review	str
	rating	int
	sentiment	int
	word_count	dict
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	int
	terrible	float
	awful	int
	wow	int
	hate	int

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+-----------+
|              name             |             review            | rating | sentiment |
+-------------------------------+-------------------------------+--------+-----------+
|     Planetwise Wipe Pouch     | it came early and was not ... |   5    |     1     |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |   5    |     1     |
| Stop Pacifier Sucking with... | This is a product well wor... |   5    |     1     |
+-------------------------------+-------------------------------+--------+-----------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'recommend': 1.0, 'disapp... |   0.0   |  0.0  |    0.0    |   0.0   | 1.0  |
| {'quilt': 1.0, 'the': 1.0,... |   0.0   |  0.0  |    0.0    |   0.0   | 0.0  |
| {'tool': 1.0, 'clever': 1.... |   0.0   |  0.0  |    0.0    |   0.0   | 2.0  |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+
| horrible | bad | terrible | awful | wow | hate |
+----------+-----+----------+-------+-----+------+
|    0     |  0  |   0.0    |   0   |  0  |  0   |
|    0     |  0  |   0.0    |   0   |  0  |  0   |
|    0     |  0  |   0.0    |   0   |  0  |  0   |
+----------+-----+----------+-------+-----+------+
[3 rows x 16 columns]

In [13]:
for word in selected_words:
    print(f'{word} = {products[word].sum()}')

awesome = 3892.0
great = 55791.0
fantastic = 1664.0
amazing = 2628.0
love = 41994.0
horrible = 1110
bad = 4183
terrible = 1146.0
awful = 687
wow = 425
hate = 1107


In [14]:
train_data,test_data = products.random_split(.8, seed=0)
features=selected_words

In [15]:
model = tc.logistic_classifier.create(train_data,target='sentiment',features=features, validation_set = test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.894671     | 0.847708          | 0.846385            |

| 5         | 6        | 2.096234     | 0.847708          | 0.846385            |

| 6         | 7        | 2.389728     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

**Weight of the words selected for modeling**

In [16]:
model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.3365913848877649,0.008929969787657008
awesome,None,1,1.1335346660341379,0.08399643983187528
great,None,1,0.8630655001196574,0.01895505244437721
fantastic,None,1,0.8858047568814237,0.11167591293399677
amazing,None,1,1.1000933113660223,0.09954776260465992
love,None,1,1.3592688669225095,0.028068300152098873
horrible,None,1,-2.2513352367590986,0.08020249388788438
bad,None,1,-0.9914778800650625,0.03848428664699068
terrible,None,1,-2.2236614360851323,0.07731736203785755
awful,None,1,-2.052908204031357,0.10099735435259254


**Evaluation of the model**

In [17]:
model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6935096220934976,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670874996,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328

## Testing with New Reviews

In [18]:
reviews = {'review1' : " I am very happy to buy this product. My baby loves this, this is pretty much awesome. Once I bought this for my elder son and he loved it very much. Amazing product!!!",
           'review2' : "this product is very bad, I hate this one. This is horrible",
           'review3' : " I loved this product once and I bought this for my neice. But this time the package was damaged. my fiance also said the same opinion \
                that the packaging is bad now a days. "
          }

In [19]:
df_item = reviews.items()
df_list = list(df_item)
df = pd.DataFrame(df_list)
df['review_num']=df[0]
df['review']=df[1]
df=df[['review_num','review']]
sf = tc.SFrame(df)

In [20]:
sf['word_count'] = tc.text_analytics.count_words(sf['review'])
for word in selected_words:
    sf[word]= sf['word_count'].apply(lambda x: count_words(x,word))
sentiment = model.predict(sf,output_type='class')
sentiment

dtype: int
Rows: 3
[1, 0, 1]